In [159]:
import matplotlib.pyplot as plt
# plt.rcParams['font.family'] = 'Times New Roman'

In [160]:

import joblib
import torch
from typing import Tuple
Tensor = torch.Tensor
from typing import Dict, Union
import numpy as np

    # default_joint_angles:
    #   left_hip_pitch_joint: -0.1
    #   left_hip_roll_joint: 0.
    #   left_hip_yaw_joint: 0.
    #   left_knee_joint: 0.3
    #   left_ankle_pitch_joint: -0.2
    #   left_ankle_roll_joint: 0.
    #   right_hip_pitch_joint: -0.1
    #   right_hip_roll_joint: 0.
    #   right_hip_yaw_joint: 0.
    #   right_knee_joint: 0.3
    #   right_ankle_pitch_joint: -0.2
    #   right_ankle_roll_joint: 0.
    #   waist_yaw_joint : 0.
    #   waist_roll_joint : 0.
    #   waist_pitch_joint : 0.
    #   left_shoulder_pitch_joint: 0.2
    #   left_shoulder_roll_joint: 0.2
    #   left_shoulder_yaw_joint: 0.
    #   left_elbow_joint: 0.9
    #   right_shoulder_pitch_joint: 0.2
    #   right_shoulder_roll_joint: -0.2
    #   right_shoulder_yaw_joint: 0.
    #   right_elbow_joint: 0.9
    
import numpy as np

default_joint_angles = np.array([
    -0.1, 0.0, 0.0, 0.3, -0.2, 0.0, 
    -0.1, 0.0, 0.0, 0.3, -0.2, 0.0, 
    0.0, 0.0, 0.0, 
    0.2, 0.2, 0.0, 0.9, 
    0.2, -0.2, 0.0, 0.9
])

def wrap_to_pi_float(angles:Union[float, np.ndarray]):
    angles %= 2*np.pi
    angles -= 2*np.pi * (angles > np.pi)
    return angles

def copysign(a, b):
    # type: (float, Tensor) -> Tensor
    a = torch.tensor(a, device=b.device, dtype=torch.float).repeat(b.shape[0])
    return torch.abs(a) * torch.sign(b)

def get_euler_xyz(q: Tensor, w_last: bool) -> Tuple[Tensor, Tensor, Tensor]:
    q=torch.from_numpy(q)
    if w_last:
        qx, qy, qz, qw = 0, 1, 2, 3
    else:
        qw, qx, qy, qz = 0, 1, 2, 3
    # roll (x-axis rotation)
    sinr_cosp = 2.0 * (q[:, qw] * q[:, qx] + q[:, qy] * q[:, qz])
    cosr_cosp = (
        q[:, qw] * q[:, qw]
        - q[:, qx] * q[:, qx]
        - q[:, qy] * q[:, qy]
        + q[:, qz] * q[:, qz]
    )
    roll = torch.atan2(sinr_cosp, cosr_cosp)

    # pitch (y-axis rotation)
    sinp = 2.0 * (q[:, qw] * q[:, qy] - q[:, qz] * q[:, qx])
    pitch = torch.where(
        torch.abs(sinp) >= 1, copysign(np.pi / 2.0, sinp), torch.asin(sinp)
    )

    # yaw (z-axis rotation)
    siny_cosp = 2.0 * (q[:, qw] * q[:, qz] + q[:, qx] * q[:, qy])
    cosy_cosp = (
        q[:, qw] * q[:, qw]
        + q[:, qx] * q[:, qx]
        - q[:, qy] * q[:, qy]
        - q[:, qz] * q[:, qz]
    )
    yaw = torch.atan2(siny_cosp, cosy_cosp)

    return wrap_to_pi_float(roll), wrap_to_pi_float(pitch), wrap_to_pi_float(yaw)


import math
def quat_to_euler(x, y, z, w):
    """Convert quaternion (xyzw) to Euler angles (roll, pitch, yaw) in radians"""
    # Roll (x-axis rotation)
    sinr_cosp = 2 * (w * x + y * z)
    cosr_cosp = 1 - 2 * (x * x + y * y)
    roll = math.atan2(sinr_cosp, cosr_cosp)

    # Pitch (y-axis rotation)
    sinp = 2 * (w * y - z * x)
    if abs(sinp) >= 1:
        print("Warning: Pitch out of range, using 90 degrees")
        pitch = math.copysign(math.pi / 2, sinp)  # Use 90 degrees if out of range
    else:
        pitch = math.asin(sinp)

    # Yaw (z-axis rotation)
    siny_cosp = 2 * (w * z + x * y)
    cosy_cosp = 1 - 2 * (y * y + z * z)
    yaw = math.atan2(siny_cosp, cosy_cosp)

    return roll, pitch, yaw


def euler_to_quat(roll, pitch, yaw):
    """Convert Euler angles (radians) to quaternion (xyzw)"""
    cy = math.cos(yaw * 0.5)
    sy = math.sin(yaw * 0.5)
    cp = math.cos(pitch * 0.5)
    sp = math.sin(pitch * 0.5)
    cr = math.cos(roll * 0.5)
    sr = math.sin(roll * 0.5)

    x = sr * cp * cy - cr * sp * sy
    y = cr * sp * cy + sr * cp * sy
    z = cr * cp * sy - sr * sp * cy
    w = cr * cp * cy + sr * sp * sy

    return np.array([x, y, z, w])

def remove_yaw(quaternions):
    """Remove yaw component by converting to Euler angles and zeroing yaw"""
    eulers = np.array([quat_to_euler(*q) for q in quaternions])
    eulers[:, 2] = 0  # Zero yaw
    new_quats = np.array([euler_to_quat(r, p, y) for r, p, y in eulers])
    return new_quats


    
import sys
from pathlib import Path

import hydra
from hydra.utils import instantiate
from hydra.core.hydra_config import HydraConfig
from hydra.core.config_store import ConfigStore
from omegaconf import OmegaConf
import omegaconf

Root_Path = Path("Please: Set the root path")
def get_motionlib_data(motion_file_path, robot_cfg_path="../humanoidverse/config/robot/g1/g1_23dof_lock_wrist.yaml"):
    robot_cfg = omegaconf.OmegaConf.load(robot_cfg_path)
    robot_cfg.robot.motion.motion_file = motion_file_path
    robot_cfg.robot.motion.asset.assetFileName = "g1_23dof_lock_wrist_fitmotionONLY.xml"
    robot_cfg.robot.motion.asset.assetRoot = Root_Path / "description/robots/g1/"
    
    from humanoidverse.utils.motion_lib.motion_lib_robot_WJX import MotionLibRobotWJX
    motion_lib = MotionLibRobotWJX(robot_cfg.robot.motion, num_envs=1, device='cpu')
    motion_data = motion_lib.load_motions(random_sample=False)[0]
    
    # print(motion_data.keys())
    # dict_keys(['global_velocity_extend', 'global_angular_velocity_extend', 'global_translation_extend', 'global_rotation_mat_extend', 'global_rotation_extend', 'global_translation', 'global_rotation_mat', 'global_rotation', 'local_rotation', 'global_root_velocity', 'global_root_angular_velocity', 'global_angular_velocity', 'global_velocity', 'dof_pos', 'dof_vels', 'fps', 'action'])
    return motion_data


def load_data(path: str)->Dict[str, np.ndarray]:
    with open(path, 'rb') as f:
        motion_data = joblib.load(f)
        assert len(motion_data) == 1, 'current only support single motion tracking'
        # get the first motion data
        motion_data = motion_data[next(iter(motion_data))]
    
    if UseMotionLib:
        motion_lib_data = get_motionlib_data(path)
        for key in motion_lib_data.keys():  
            
            # print(key, type(motion_lib_data[key]))
            if not key in motion_data:
                motion_data[key] = motion_lib_data[key].numpy()
        ...
    
    
    if not 'motion_times' in motion_data:
        print("Warning: motion_times not in motion_data, calculating it from fps")
        fps = motion_data['fps']
        motion_data['motion_times'] = (np.arange(motion_data['dof'].shape[0])+1) / fps
    motion_data['frame_idx'] = motion_data['motion_times']*motion_data['fps']
    
    motion_data['rpy'] = torch.stack(get_euler_xyz(motion_data['root_rot'],True),dim=1).numpy()
    motion_data['root_rot_rmyaw'] = remove_yaw(motion_data['root_rot'])
    assert np.allclose(get_euler_xyz(motion_data['root_rot_rmyaw'],True)[2], np.zeros_like(motion_data['root_rot_rmyaw'][:,2]), atol=1e-6), f"Yaw should be zero, got {get_euler_xyz(motion_data['root_rot_rmyaw'],True)[2]}"
    
    if 'action' in motion_data:
        motion_data['target_dof'] = motion_data['action']*0.25 + default_joint_angles.reshape(1,-1)
        
    if Roll2 and 'metrics' in path:
        key = 'motion_times'
        motion_data[key] = np.roll(motion_data[key], -2, axis=0)
        for k in motion_data.keys():    
            if isinstance(motion_data[k], np.ndarray):
                motion_data[k] = motion_data[k][0:-3]
    print('motion_times: ', motion_data['motion_times'])
        
    print('motion_data keys: ', {k:v.shape if isinstance(v, np.ndarray) else v for k,v in motion_data.items()})
    # dict_keys(['root_trans_offset', 'pose_aa', 'dof', 'root_rot', 'actor_obs', 'action', 'terminate', 'root_lin_vel', 'root_ang_vel', 'dof_vel', 'motion_times', 'fps'])
    return motion_data

# mot_data = load_data(mot_path)







In [161]:
Roll2=True
UseMotionLib=True

In [ ]:


# Example
# motion_data_dict = {
#     'pc': ".../PBHC/logs_0530/20250422_024844-ISA_Fight_I5E-3_Fix-motion_tracking-g1_23dof_lock_wrist/motions/PC_normal_344f.pkl",
#     'nx': ".../PBHC/logs_0530/20250422_024844-ISA_Fight_I5E-3_Fix-motion_tracking-g1_23dof_lock_wrist/motions/NX_explode_76f.pkl"
# }

motion_data_dict = {
    'empty': "",
}

motion_data_dict = {
    'empty': "",
}
motion_data_dict = {k:load_data(v) for k,v in motion_data_dict.items()}


In [166]:
import matplotlib.pyplot as plt
# clipto = [560, 586]
# clipto = [0,70]
clipto = [1000,-1]
# clipto=[500,650]

def simple_visualize(motion_data_dict, x_key, y_key, x_dim=0, y_dim=0):
    """
    简单可视化函数
    
    参数:
        motion_data_dict: 运动数据字典 {name: data}
        x_key: x轴数据key
        y_key: y轴数据key
        x_dim: x数据的维度索引
        y_dim: y数据的维度索引
    """
    plt.figure(figsize=(10, 6))
    for name, motion_data in motion_data_dict.items():
        
        x_data = motion_data[x_key]
        y_data = motion_data[y_key]
        
        len_x = len(x_data)
        len_y = len(y_data)
        
        # 处理不同维度的数据
        if x_data.ndim == 1:
            x = x_data
        elif x_data.ndim == 2 :
            x = x_data[:, x_dim] 
        elif x_data.ndim > 2:
            x = x_data.reshape(len_x,-1)[:, x_dim]
            print(f"Warning: x_data has more than 2 dimensions, reshaping to {x_data.reshape(len_x,-1).shape}")
        
        if y_data.ndim == 1:
            y = y_data
        elif y_data.ndim >= 2:
            y = y_data.reshape(len_y,-1)[:, y_dim] 
            if y_data.ndim > 2:
                print(f"Warning: y_data has more than 2 dimensions, reshaping to {y_data.reshape(len_y,-1).shape}")
        
        plt.plot(x[clipto[0]:clipto[1]], y[clipto[0]:clipto[1]], label=name)
        plt.xlabel(f"{x_key} (dim {x_dim})")
        plt.ylabel(f"{y_key} (dim {y_dim})")
        plt.title(f"{y_key} vs {x_key}")
        plt.grid(True)
        plt.legend()
    plt.show()

# 使用示例


# simple_visualize(motion_data_dict, 'motion_times', 'root_rot_rmyaw', x_dim=0, y_dim=0)
# simple_visualize(motion_data_dict, 'motion_times', 'root_rot_rmyaw', x_dim=0, y_dim=1)
# simple_visualize(motion_data_dict, 'motion_times', 'root_rot_rmyaw', x_dim=0, y_dim=2)
# simple_visualize(motion_data_dict, 'motion_times', 'root_rot_rmyaw', x_dim=0, y_dim=3)

# simple_visualize(motion_data_dict, 'motion_times', 'dof', x_dim=0, y_dim=0)
# simple_visualize(motion_data_dict, 'motion_times', 'dof', x_dim=0, y_dim=1)

In [167]:
import matplotlib.pyplot as plt
import numpy as np
import os
from math import ceil, sqrt
from typing import List
from matplotlib.backends.backend_pdf import PdfPages

def visualize_all_keys_all_dims(
    motion_data_dict,
    clipto=[1000, -1],
    save_path="all_motion_keys_plot.pdf",
    plots_per_page=20,
    keys: List[str] = None,
):
    """
    可视化 motion_data_dict 中指定 keys 的所有维度，按网格排列并保存为一个多页 PDF。

    参数:
        motion_data_dict: {name: {key: ndarray}}
        clipto: 裁剪范围 [start, end]
        save_path: 多页 PDF 保存路径
        plots_per_page: 每页子图数
        keys: 选择性绘制的 key 列表（默认全部）
    """
    available_keys = sorted({k for d in motion_data_dict.values() for k in d.keys()})
    if keys is None:
        selected_keys = available_keys
    else:
        selected_keys = [k for k in keys if k in available_keys]
        missing = [k for k in keys if k not in available_keys]
        if missing:
            print(f"⚠️ 警告: 下列 key 不在数据中，将被跳过: {missing}")

    plot_tasks = []
    for key in selected_keys:
        for data in motion_data_dict.values():
            if key in data:
                sample = data[key]
                break
        else:
            continue
        dims = 1 if sample.ndim == 1 else sample.reshape(len(sample), -1).shape[1]
        for dim in range(dims):
            plot_tasks.append((key, dim))

    total_pages = ceil(len(plot_tasks) / plots_per_page)
    print(f"📄 选中图数：{len(plot_tasks)}，将生成 {total_pages} 页")

    os.makedirs(os.path.dirname(save_path) or ".", exist_ok=True)
    with PdfPages(save_path) as pdf:
        for page in range(total_pages):
            num_subplots = min(plots_per_page, len(plot_tasks) - page * plots_per_page)
            ncols = ceil(sqrt(num_subplots))
            nrows = ceil(num_subplots / ncols)

            fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3 * nrows), squeeze=False)
            axes = axes.flatten()

            for i in range(len(axes)):
                ax = axes[i]
                task_idx = page * plots_per_page + i
                if task_idx >= len(plot_tasks):
                    ax.axis('off')
                    continue

                key, dim = plot_tasks[task_idx]
                for name, motion_data in motion_data_dict.items():
                    if key not in motion_data:
                        continue
                    y_data = motion_data[key]
                    len_y = len(y_data)
                    x_data = motion_data.get("motion_times", np.arange(len_y))

                    # print(key,dim)
                    x = x_data.reshape(len(x_data), -1)[:, 0] if x_data.ndim > 1 else x_data
                    y = y_data.reshape(len_y, -1)[:, dim] if y_data.ndim > 1 else y_data

                    ax.plot(x[clipto[0]:clipto[1]], y[clipto[0]:clipto[1]], label=name)

                ax.set_title(f"{key}[{dim}]", fontsize=8)
                ax.tick_params(labelsize=6)
                ax.grid(True)
                ax.legend(fontsize=6)

            plt.tight_layout()
            pdf.savefig(fig)
            plt.close()
            print(f"✅ 第 {page + 1} 页加入 PDF")

    print(f"🎉 所有页已合并保存为：{save_path}")


In [ ]:
visualize_all_keys_all_dims(
    motion_data_dict,
    clipto=[0, 200],
    save_path="final_motion_plots.pdf",
    plots_per_page=23,
    keys=["dof", "dof_vel", "tau", 'target_dof', 'rpy', "root_ang_vel"]
)
